In [8]:
import numpy as np
import matplotlib.pyplot as plt

from create_prepared_data_tsfresh import *
from compression import calculateCompRatio, compress_dataset
from utils.personal_utils import *

import os
import pywt

import matplotlib.pyplot as plt
import time

In [9]:
all_ds = {
    'AppliancesEnergy':   '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts',
    'NewsTitleSentiment': '/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TEST.ts',
    'BenzeneConcentration':'/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TEST.ts',
    'BeijingPM25Quality': '/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TEST.ts',

    'IEEEPPG':            '/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts',
    'FloodModeling1':     '/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TEST.ts',
    'HouseholdPowerConsumption1': '/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TEST.ts',
    'Covid3Month':              '/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TEST.ts'
}

Steps:

Keep in mind: 
- Ignore 0.99 for now, to have more overlapping features.
- Selected features come from train_set and are then same for test_set
- Correlation Analysis on TRAIN Set: Same as for TEST, just to get y value let the pkl. model run and get the target value!
- When redoing runs, consider keeping more features! (Maybe no AutoMl featuere selectin! Or analyse both!)
- I assume the rows in the logged prediction and loaded df are in the same order. -> but should be! -> if not error wouldn't make sense


Found out: 
- Removing None or 0.99 or anything else gives bach a few features for each ds. But not really much.
-> Can consider removing 0.99 from comparing. Too much compression different features.
- The dim are already in the .txt file not in order!
- Both functions seem to work. Simple test.
- Also no overlapping features for comp_algos if ignoring Appliances



In [10]:
# Get the overlapping features for each (Dataset, Technique)
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'AppliancesEnergy',
    'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month'
]

tqs = ['dct', 'dft', 'dwt']

common_features_dict = {}

# Helper function to find common elements in lists
def common_elements(lists):
    if not lists:
        return []
    common = lists[0]
    for lst in lists[1:]:
        common = [elem for elem in common if elem in lst]
    return common

# Iterate over them to go for each value
for ds_name in ds_names:


    for tq in tqs:
        path_to_runs = log_path + f'{ds_name}_Runs/'
        common_features = None
        features_lists = []
        
        for dval in dropout_values:
            if dval is None:
                path_to_selectedf = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/selected_features.txt'
            else:
                path_to_selectedf = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/selected_features.txt'
            
            try:
                with open(path_to_selectedf, 'r') as file:
                    lines = file.read().splitlines()  # Read and store lines in a list
                    features_lists.append(lines)
                        
            except FileNotFoundError:
                print(f"File {path_to_selectedf} not found, skipping.")
                continue
        
        common_features = common_elements(features_lists)
        common_features_dict[(ds_name, tq)] = common_features





# Print overlapping for (Dataset,Comp_Alg) combinaton
for (ds_name, tq), common_features in common_features_dict.items():
    if common_features is not None:
        print(f"Number of overlapping features for {ds_name}: {tq} {len(common_features)}")
print()



# Print num of overlapping for each tq
overlap_per_tq = {}
for tmp_tq in tqs:
        overlap_per_tq[tmp_tq] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if tq == tmp_tq])
        print(f"Number of overlapping features for whole {tmp_tq} {len(overlap_per_tq[tmp_tq])}")
print()


# Print number of overlapping for each dataset
overlapp_per_dataset = {}
for tmp_dsname in ds_names:

        overlapp_per_dataset[tmp_dsname] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if ds_name == tmp_dsname])
        print(f"Number of overlapping features for whole {tmp_dsname} {len(overlapp_per_dataset[tmp_dsname])}")

Number of overlapping features for AppliancesEnergy: dct 4
Number of overlapping features for AppliancesEnergy: dft 1
Number of overlapping features for AppliancesEnergy: dwt 2
Number of overlapping features for NewsTitleSentiment: dct 78
Number of overlapping features for NewsTitleSentiment: dft 63
Number of overlapping features for NewsTitleSentiment: dwt 122
Number of overlapping features for BenzeneConcentration: dct 230
Number of overlapping features for BenzeneConcentration: dft 208
Number of overlapping features for BenzeneConcentration: dwt 235
Number of overlapping features for BeijingPM25Quality: dct 81
Number of overlapping features for BeijingPM25Quality: dft 80
Number of overlapping features for BeijingPM25Quality: dwt 90
Number of overlapping features for IEEEPPG: dct 151
Number of overlapping features for IEEEPPG: dft 117
Number of overlapping features for IEEEPPG: dwt 108
Number of overlapping features for FloodModeling1: dct 29
Number of overlapping features for FloodM

In [11]:
# Calculate the correlation, we get a dict looking like: correlations[('AppliancesEnergy', 'dft', 0.5)] -> pd. of correlation
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month',
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
]


tqs = ['dct', 'dft', 'dwt']

prepared_dfs = {}
correlations = {}

for ds_name in ds_names:
    path_to_runs = log_path + f'{ds_name}_Runs/'



    df_raw = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
    #df_raw = load_and_prepare_everything(all_ds[ds_name], None, None)[1].drop('target', axis=1)
    
    path_to_pred = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/predictions.npy'
    prediction_raw = np.load(path_to_pred)


    for tq in tqs:
        ovf_list = common_features_dict[(ds_name, tq)]

        df_raw_tq = df_raw[ovf_list].copy() # Select overlapping features
        df_raw_tq['prediction'] = prediction_raw

        prepared_dfs[(ds_name,tq,None)] = df_raw_tq.copy() # bc different features for each technique




        for dval in dropout_values[1:]:
            

            features = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
            #features = load_and_prepare_everything(all_ds[ds_name], tq, dval)[1].drop('target', axis=1)
            
            
            ovf_df = features[ovf_list]
            path_to_pred = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/predictions.npy'

            try:
                with open(path_to_pred, 'r') as file:
                    prediction = np.load(path_to_pred)
            except FileNotFoundError:
                print(f"File {path_to_pred} not found, skipping.")
                continue

            ovf_df['prediction'] = prediction
            prepared_dfs[(ds_name,tq, dval)] = ovf_df.copy()


            # Add Subtraction and Correlation
            sub_df = (df_raw_tq - ovf_df)
            corrs = sub_df.corrwith(sub_df['prediction'])
            correlations[(ds_name,tq,dval)] = corrs
            


In [12]:
# Get Average Correlatin of absolute value over each (ds_name, tq)
# The Entries are Series object, the row name is the featuer name!
corr_ds_name_tq = {}


for ds_name in ds_names:
    

    for tq in tqs:

        # Initalize a df with same columns, rows but zero in each field
        avg_corr = correlations[(ds_name,tq,0.5)].copy()
        avg_corr[:] = 0
        #print(avg_corr)

        for d_val in dropout_values[1:]:
            curr_corr_df = correlations[(ds_name,tq,d_val)].abs()  # take abs value
            avg_corr = curr_corr_df + avg_corr
        
        avg_corr = avg_corr / len(dropout_values[1:])
    
        
        corr_ds_name_tq[(ds_name, tq)] = avg_corr.sort_values(ascending=False)



corr_ds_name = {}
for ds_name in ds_names:
    corr_ds_name[ds_name] = corr_ds_name_tq[(ds_name, 'dct')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dft')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dwt')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] = corr_ds_name[ds_name].sort_values(ascending=False) / 3
    




In [17]:
# Get Correlation over each dataset

for ds_name in ds_names:
    print(ds_name )
    print(corr_ds_name[(ds_name)])
    print()


FloodModeling1
dim_1__agg_linear_trend__attr_rvalue__chunk_len_50__f_agg_mean     0.096644
dim_1__agg_linear_trend__attr_stderr__chunk_len_50__f_agg_mean     0.070737
dim_1__linear_trend__attr_pvalue                                   0.065415
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_2    0.063661
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_1    0.060218
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_8    0.057857
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_0    0.057480
dim_1__augmented_dickey_fuller__attr_teststat__autolag_AIC         0.047607
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_5    0.042560
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_4    0.037384
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_6    0.036943
dim_1__fft_coefficient__attr_angle__coeff_12                       0.029481
dtype: float64

HouseholdPowerConsumption1
dim_2__longest_strike_above_me

In [14]:
# Print for Each Combination the top 5 features
for ds_name in ds_names:
    for tq in tqs:
        print(ds_name + " " + tq)
        print(corr_ds_name_tq[(ds_name, tq)].head(6))
        print()



FloodModeling1 dct
prediction                                                         1.000000
dim_1__fourier_entropy__bins_100                                   0.174033
dim_1__fft_coefficient__attr_abs__coeff_1                          0.100530
dim_1__agg_linear_trend__attr_stderr__chunk_len_10__f_agg_var      0.096744
dim_1__fft_coefficient__attr_abs__coeff_17                         0.082800
dim_1__energy_ratio_by_chunks__num_segments_10__segment_focus_2    0.078819
dtype: float64

FloodModeling1 dft
prediction                                                        1.000000
dim_1__agg_linear_trend__attr_stderr__chunk_len_50__f_agg_var     0.163986
dim_1__agg_linear_trend__attr_rvalue__chunk_len_50__f_agg_mean    0.150673
dim_1__fft_coefficient__attr_abs__coeff_13                        0.130866
dim_1__fft_coefficient__attr_angle__coeff_24                      0.118444
dim_1__linear_trend__attr_pvalue                                  0.093995
dtype: float64

FloodModeling1 dwt
predi

In [15]:
# To sort features

import re
def extract_fifth_number(column_name):
    match = re.match(r'^.{4}(\d+)', column_name)  # Match numbers after the first 4 characters
    if match:
        return int(match.group(1))
    return float('inf')  # Use infinity for columns without a valid number

# Create a dictionary with column names as keys and extracted numbers as values
column_sort_order = {col: extract_fifth_number(col) for col in features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
features_sorted = features[sorted_columns]
features_sorted

,dim_1__variance_larger_than_standard_deviation,dim_1__has_duplicate_max,dim_1__has_duplicate_min,dim_1__has_duplicate,dim_1__sum_values,dim_1__abs_energy,dim_1__mean_abs_change,dim_1__mean_change,dim_1__mean_second_derivative_central,dim_1__median,...,dim_5__fourier_entropy__bins_10,dim_5__fourier_entropy__bins_100,dim_5__permutation_entropy__dimension_3__tau_1,dim_5__permutation_entropy__dimension_4__tau_1,dim_5__permutation_entropy__dimension_5__tau_1,dim_5__permutation_entropy__dimension_6__tau_1,dim_5__permutation_entropy__dimension_7__tau_1,dim_5__query_similarity_count__query_None__threshold_0.0,dim_5__mean_n_absolute_max__number_of_maxima_7,target
0,0.0,0.0,0.0,0.0,-13.328921,713.695651,0.075384,-0.000040,1.083949e-06,-0.156129,...,0.204871,0.519138,0.973845,1.281895,1.595168,1.907794,2.212136,0.0,2.679796,62.696
1,0.0,0.0,0.0,0.0,33.786040,708.899485,0.063141,-0.000024,-1.150174e-06,-0.178968,...,0.200814,0.474437,1.056601,1.456673,1.859125,2.223844,2.568662,0.0,1.990376,60.206
2,0.0,0.0,0.0,0.0,-6.254906,745.563479,0.057998,0.000031,-3.361054e-05,-0.259542,...,0.254093,0.578736,1.018086,1.370997,1.725431,2.075031,2.408576,0.0,4.562132,59.055
3,0.0,0.0,0.0,0.0,3.575914,715.433222,0.062068,-0.000239,-2.213488e-05,-0.176962,...,0.215617,0.339942,0.910026,1.174233,1.453594,1.732195,1.991041,0.0,4.783811,59.389
4,0.0,0.0,0.0,0.0,-31.364350,684.421410,0.076260,-0.000299,-3.613121e-05,-0.097175,...,0.226363,0.395585,0.925202,1.208734,1.508511,1.809548,2.097549,0.0,5.245195,59.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,0.0,0.0,0.0,0.0,-41.662881,861.403352,0.062652,0.000366,3.105676e-05,-0.270395,...,0.260704,0.493589,0.973395,1.282462,1.591754,1.892459,2.175470,0.0,3.768368,81.967
1324,0.0,0.0,0.0,0.0,35.670370,910.130878,0.057120,-0.000092,9.445828e-07,-0.203987,...,0.260704,0.620602,0.998149,1.350227,1.715519,2.065704,2.398914,0.0,3.293080,82.317
1325,0.0,0.0,0.0,0.0,-0.807496,924.786788,0.060533,0.000035,4.490620e-08,-0.276969,...,0.249958,0.406332,0.933321,1.194502,1.464577,1.730555,1.996039,0.0,2.617577,83.241
1326,0.0,0.0,0.0,0.0,25.649074,956.614626,0.058234,-0.000050,-2.119782e-06,-0.220346,...,0.249958,0.440417,0.977677,1.290420,1.616200,1.923172,2.224485,0.0,2.195226,83.927


In [16]:
file_path = '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

selected_features = features[lines]


column_sort_order = {col: extract_fifth_number(col) for col in selected_features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
sfeatures_sorted = selected_features[sorted_columns]
sfeatures_sorted

FileNotFoundError: [Errno 2] No such file or directory: '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'